# Overview
__In this notebook, we'll:__
1. Look at an example of how to apply concepts from lecture to output from a "stochastic climate model"
2. Apply concepts from lecture to output from CMIP data

__Conceptual learning objectives:__
- Understand difference between externally-forced and internal climate variability.
- Understand why we need ensembles of climate simulations.

#### specify path to data

In [ ]:
## filepath for the mount
cmip6_fp = "/Volumes/cmip6/data"

#### Import packages

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cmocean
from matplotlib.dates import DateFormatter
import seaborn as sns
import glob
import cftime
import src.utils
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import xesmf as xe

## set plotting style
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## initialize random number generator
rng = np.random.default_rng()

#### Simulate synthetic climate

In [ ]:
## specify number of ensemble members and end year for simulation
n_members = 1000
tf = 2006

## simulation pre-industrial and warming scenarios
T_PI = src.utils.markov_simulation(ti=1850, tf=tf, n_members=n_members, trend=0)
T_warming = src.utils.markov_simulation(
    ti=1850, tf=tf, n_members=n_members, trend=0.005
)

## for convenience, get subset of pre-industrial control which overlaps with warming
T_PI_hist = T_PI.sel(year=T_warming.year)

#### Plot a random sample from each simulation

In [ ]:
# choose a random sample
idx = rng.choice(T_PI.ensemble_member)

# make the plot
fig, ax = plt.subplots(figsize=(4, 3))

ax.plot(
    T_PI_hist.year,
    T_PI_hist.sel(ensemble_member=idx),
    color="black",
    label="P.I. control",
)
ax.plot(
    T_warming.year, T_warming.sel(ensemble_member=idx), color="red", label="warming"
)

## label axes
ax.set_xlabel("Year")
ax.set_ylabel(r"SST anomaly ($^{\circ}C$)")
ax.legend(prop={"size": 10})
ax.set_title("random ensemble member")

plt.show()

#### Plot ensemble mean and spread

In [ ]:
def plot_ensemble_spread(ax, T, color, label=None):
    """plot mean and +/- 1 standard dev. of ensemble on
    given ax object."""

    ## compute stats
    mean = T.mean("ensemble_member")
    std = T.std("ensemble_member")

    ## plot mean
    mean_plot = ax.plot(mean.year, mean, label=label, color=color)

    ## plot spread
    ax.plot(mean.year, mean + std, lw=0.5, c=mean_plot[0].get_color())
    ax.plot(mean.year, mean - std, lw=0.5, c=mean_plot[0].get_color())

    return


## Plot ensemble stats
fig, ax = plt.subplots(figsize=(4, 3))

## plot data
plot_ensemble_spread(ax, T_PI_hist, color="black", label="P.I. control")
plot_ensemble_spread(ax, T_warming, color="red", label="warming")

## label axes
ax.set_xlabel("Year")
ax.set_ylabel(r"SST anomaly ($^{\circ}C$)")
ax.legend(prop={"size": 10})
ax.set_title("Ensemble results")

plt.show()

#### Is the trend significant?
- histogram for PI control (generate by computing trend for 1,000 random 40-year segments)
- compare to trend in warming simulation over last 40 years and for period 1900-1950

In [ ]:
def get_slope(data, dim="year"):
    """Function to compute linear trend of SST,
    in deg/century."""

    ## fit linear trend to data
    coefs = data.polyfit(dim=dim, deg=1)["polyfit_coefficients"]

    ## Get slope (degree=1; intercept is given by degree=0).
    ## Note: units are in deg/year
    slope = coefs.sel(degree=1)

    ## convert units to deg/century
    slope *= 100

    return slope


def plot_histogram_comparison(ax, samples0, samples1, label0=None, label1=None):
    """
    Compute two histograms, one each for samples0 and samples1.
    Plot the results on the specified ax object, and label histograms
    'label0' and 'label1', respectively.
    """

    ## First, make the histograms.
    # specify histogram bins
    bin_width = 0.5
    bin_edges = np.arange(-4.75, 4.75 + bin_width, bin_width)

    # compute histograms
    hist0, _ = np.histogram(samples0, bins=bin_edges)
    hist1, _ = np.histogram(samples1, bins=bin_edges)

    ## plot histograms
    ax.stairs(values=hist0, edges=bin_edges, color="k", label=label0)
    ax.stairs(
        values=hist1,
        edges=bin_edges,
        color="r",
        label=label1,
        fill=True,
        alpha=0.3,
    )

    ## label plot
    ax.set_ylabel("Count")
    ax.set_xlabel(r"Warming trend ($^{\circ}C~/~$century)")

    return ax


def plot_histogram_comparison_wrapper(ax, years):
    """wrapper function to plot histogram comparison for given subset of years"""

    ## Get trends for each ensemble members
    T_PI_subset = T_PI.sel(year=years)
    T_warming_subset = T_warming.sel(year=years)
    trends_PI = get_slope(T_PI_subset)
    trends_warming = get_slope(T_warming_subset)

    ## make the plot
    ax = plot_histogram_comparison(
        ax,
        samples0=trends_PI,
        samples1=trends_warming,
        label0="PI control",
        label1="warming",
    )

    ## plot ensemble means
    T_PI_mean = T_PI_subset.mean("ensemble_member")
    T_warming_mean = T_warming_subset.mean("ensemble_member")
    ax.axvline(get_slope(T_PI_mean), ls="--", c="k", lw=1)
    ax.axvline(get_slope(T_warming_mean), ls="--", c="r", lw=1)

    return ax

In [ ]:
## Make plot
fig, axs = plt.subplots(1, 2, figsize=(8, 3))

## Plot data for each subset of years
axs[0] = plot_histogram_comparison_wrapper(axs[0], years=np.arange(1850, 1890))
axs[1] = plot_histogram_comparison_wrapper(axs[1], years=np.arange(1966, 2006))

## label plot
axs[0].set_title("First 40 years")
axs[1].set_title("Last 40 years")
axs[0].legend(prop={"size": 10})
axs[1].set_yticks([])
axs[1].set_ylabel(None)
axs[1].set_ylim(axs[0].get_ylim())

plt.show()

# Now it's your turn...
Start by picking a variable/index to analyze (e.g., SST at a specific location or averaged over a specified region), a reanalysis product from the CMIP archive, and a model from the CMIP archive.

#### Model validation
1. Load the data and make a plot.
2. Compute the index for both the reanalysis and the model (in the historical simulation).
3. How do the statistics compare over the overlapping period? (e.g., mean, standard deviation, seasonal cycle, power spectrum).

#### Is there evidence of a climate change signal?
1. Next, compute the index in the model's pre-industrial control simulation (__should this be a last millenium simulation?).
2. Create a histogram of trends in the control simulation by computing the trends of randomly-sampled 40-year segments.
3. Based on this histogram, is the model-simulated trend (from the last 40 years, in the historical simulation) significant?
   
#### What are the projected future changes?
1. Next, compute the index in a future warming scenario (using the same model).
2. How do the statistics/histogram shift in the future warming scenario?